In [82]:
import re
import spacy
from spacy_sentiws import spaCySentiWS
import os
nlp = spacy.load('de')
sentiws = spaCySentiWS(sentiws_path=os.path.join(os.getcwd(), 'aux'))
nlp.add_pipe(sentiws)

In [83]:
input_path = os.path.join(os.getcwd(), "input")
with open(os.path.join(input_path, "positive_example2.txt"), "r") as f:
     text = f.read()  
        

In [84]:
def clean_text(text, substitution_dict):
    for key, value in sub_dict.items():
        text = re.sub(key, value, text)
    return text

In [85]:
# Define dictionary with patterns to substitute for cleaning purposes
sub_dict = {
    "\n\n": "\n",
}

In [86]:
text = clean_text(text, sub_dict)

In [87]:
doc = nlp(text)

In [88]:
targets = ["flüchtlinge", "flüchtling", "geflüchtete", "geflüchteter", "geflüchtete", "asylbewerber", "asylsuchender", "asylant"]

In [89]:
def score_sentences(doc, targets):
    """Score sentences that contain any of the target patterns. Returns a list of tuples"""
    compiled_targets = [re.compile(target) for target in targets]
    results = []
    for sentence in doc.sents:
        if any(target.search(sentence.lower_) for target in compiled_targets):
            # I exclude "Flüchtlinge" and synonyms because they are scored as negative sentiment in the model
            sentiments = [token._.sentiws for token in sentence if token._.sentiws and token.lower_ not in targets]
            if len(sentiments) > 0:
                score = sum(sentiments) / len(sentiments)
                results.append((sentence, score))
    return results

In [90]:
# TODO: It is probably good to give each article an ID in my collection so that I don't have to save the whole doc with the score.
def score_articles(doc):
    results = []
    sentiments = [token._.sentiws for token in doc if token._.sentiws and token.lower_]
    if len(sentiments) > 0:
        score = sum(sentiments) / len(sentiments)
        results.append((doc.text[:100], score))
    return results

In [91]:
score_sentences(doc, targets)

[(Gefährliche Islamisten dürfen bleiben, brave Flüchtlinge müssen gehen.,
  -0.6366)]

In [92]:
score_articles(doc)

[('Abschiebegrund: Pünktlichkeit und Regeltreue\nEine Kolumne von Jan Fleischhauer\nWir schieben oft die ',
  -0.022496296296296302)]